<a href="https://colab.research.google.com/github/RFajardoMonzon/MachineLearningCourse/blob/master/Overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 16 - Overfitting.


*   Recuerda que puedes consultar la documentación sobre una función escribiendo **?** justo después de la función: *Ejemplo: np.maximum?*
*   Puedes ejecutar el contenido de una celda con el atajo de teclado **CTRL+ENTER**
*   Utiliza **TAB** cada vez que quieras autocompletar una llamada a una función.
*   Puedes ejecutar instrucciones de bash directamente desde el notebook usando **!** : *Ejemplo: !pip install tensorflow*
*   Recuerda que Google es tu amigo, y saber buscar la información en las documentaciones de las librerías es muy importante.
*   Una solución correcta no es la que funciona sino la que se entiende!
*   No dudes en preguntar cualquier duda al profesor que lleva todo el día dando la turra.

## 1. ¿Esta opinión es tuya o de otro?


Hoy vamos a trabajar  nuevamente con el dataset  de reviews de películas recopilas de la web IMDB. Ayer entrenamos a nuestro clasificador para que a partir de una revie, supiera predecir si el sentimiento asociado a esa opinión era *positiva* o *negativa*. Ayer conseguimos, combinando diferentes arquitecturas, que nuestro clasificador pudiera predecir con una precisión superior al 80% en el set de validación.

> > > > > <img src=https://www.samyzaf.com/ML/imdb/review1.png width=500px>

 Sin embargo, esta cifra rápidamente se deterioraba con la aparición del ***overfitting*** en nuestro modelo. Hoy probaremos diferentes técnicas de ***regularización*** para tratar de solucionar esto.
 
 1. Re-entrena el modelo que diseñaste ayer (*si no lo tienes, comienza trabajando en este*). Entrena durante varias ***epochs*** (e.g. epochs=20) y registra los diferentes valores de *loss* alcanzado por el modelo durante cada *epoch* para el train y test set. Una vez lo tengas, representalos en una misma gráfica y evalua si hay existencia de overfitting. Después, haz uso del EarlyStopping para que tu modelo pare automáticamente en presencia del overfitting.***(Bonus)*** Puedes probar a visualizarlo en tiempo real a través de tensorboard, o configurando tu propio *callback*()
 
 2.  Conjuntamente prueba a entrenar a un modelo con mayor capacidad, y a otros modelo con menor capacidad. Grafícalos conjuntamente de manera que se pueda observar los fenómenos del ***underfitting()*** y el ***overfitting()***. Utiliza corréctamente las leyendas para identificar a cada modelo.
 
 3. Finalmente repite el experimento para diferentes configuraciones de redes, que hagan uso del **Dropout**, **BatchNormalization** y **Weight Decay**, con diferentes valores de sus parámetros. Obtén al menos 9 combinaciones diferentes de entrenamientos y represéntalos conjuntamente en un gráfico. (dada la densidad de ***plots*** que puede haber, en este apartado no representes el ***training_loss***)

In [0]:
import numpy as np
old = np.load

np.load = lambda *a,**k: old(*a, allow_pickle=True, **k)

In [0]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
import matplotlib.pyplot as plt

max_features = 10000 # Número máximo de palabras diferentes de nuestro vocabulario.
maxlen = 300         # Número máximo de palabras en cada texto.

# Carga el texto ya tokenizado.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features) 

# Preprocesado de los textos para igualar todas sus longitudes.
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test =  preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen) 

# Utiliza esta línea para obtener la palabra asociada a cada índice.
# imdb.get_word_index()
nModel = 1

17465344/17464789 [==============================] - 0s 0us/step


In [0]:
model = Sequential()

model.add(Embedding(max_features, 128))

model.add(CuDNNLSTM(128))

model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer=SGD(1.5), loss="binary_crossentropy", metrics=["acc"])

history = model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

model.save("LSTM{}.h5".format(nModel))

nModel += 1

Instructions for updating:
Colocations handled automatically by placer.
Train on 25000 samples, validate on 25000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
25000/25000 [==============================] - 21s 830us/sample - loss: 0.6849 - acc: 0.5381 - val_loss: 0.6050 - val_acc: 0.6554
Epoch 2/20
25000/25000 [==============================] - 17s 690us/sample - loss: 0.6075 - acc: 0.6644 - val_loss: 0.5239 - val_acc: 0.7380
Epoch 3/20
25000/25000 [==============================] - 17s 679us/sample - loss: 0.4497 - acc: 0.7898 - val_loss: 0.3945 - val_acc: 0.8235
Epoch 4/20
25000/25000 [==============================] - 17s 681us/sample - loss: 0.3132 - acc: 0.8681 - val_loss: 0.2812 - val_acc: 0.8840
Epoch 5/20
25000/25000 [==============================] - 17s 681us/sample - loss: 0.2373 - acc: 0.9057 - val_loss: 0.3502 - val_acc: 0.8574
Epoch 6/20
25000/25000 [==============================] - 17s 686us/sample - loss: 0.1815 - acc: 0.9314 - val_loss: 0.2776 - 

In [0]:
# model = load_model("LSTM1.h5")
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = history.epoch

plt.plot(epochs, loss, label="loss")
plt.plot(epochs, val_loss, label="val_loss")
plt.legend()
plt.show()

In [0]:
from tensorflow.keras.callbacks import EarlyStopping

callback = [EarlyStopping("val_loss", patience=5)]

model_ES = Sequential()

model_ES.add(Embedding(max_features, 128))

model_ES.add(CuDNNLSTM(128))

model_ES.add(Dense(1, activation="sigmoid"))

model_ES.compile(optimizer=SGD(1.5), loss="binary_crossentropy", metrics=["acc"])

history_ES = model_ES.fit(x_train, y_train, epochs=20,
                       validation_data=(x_test, y_test), callbacks=callback)

model_ES.save("LSTM_ES{}.h5".format(nModel))

nModel += 1

In [0]:
loss_ES = history_ES.history["loss"]
val_loss_ES = history_ES.history["val_loss"]
epochs_ES = history_ES.epoch

plt.plot(epochs_ES, loss_ES, label="loss")
plt.plot(epochs_ES, val_loss_ES, label="val_loss")
plt.legend()
plt.show()

In [0]:
model_uf = Sequential()


model_uf.add(Embedding(max_features, 4, input_shape=(maxlen,), trainable=False))

# model_fc.summary()

model_uf.add(CuDNNLSTM(4))

model_uf.add(Dense(1, activation="sigmoid"))

model_uf.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

history_uf = model_uf.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

model_uf.save("LSTM_uf{}.h5".format(nModel))

nModel += 1

In [0]:
mini_x_train = x_train[:1000]
mini_y_train = y_train[:1000]

mini_x_test = x_test[:1000]
mini_y_test = y_test[:1000]

In [0]:
from tensorflow.keras.backend import clear_session

history = []

for i in np.arange(15):

  clear_session()
  
  title = ""

  model = Sequential()

  model.add(Embedding(max_features, 32))

  model.add(CuDNNLSTM(64))

  if i >= 5: 
    title += "Dropout 1 = {}".format(i%5 / 10)
    model.add(Dropout(i%5 / 10))

  model.add(Dense(64))

#   model.add(BatchNormalization())

  model.add(Activation("relu"))

  if i < 5 or i >= 10:
    title += "Dropout 2 = {}".format(i%5 /10)
    model.add(Dropout(i%5 / 10))

  model.add(Dense(1, activation="sigmoid"))

  model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

  hist = model.fit(mini_x_train, mini_y_train, epochs=20, 
                           validation_data=(mini_x_test, mini_y_test))

  model.save("LSTM_DO{}.h5".format(nModel))

  nModel += 1
  
  val_loss = hist.history["val_loss"]
  epochs = hist.epoch
  history.append(hist)

  plt.plot(epochs, val_loss, label=title)

plt.legend()
plt.show()